In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, KFold



In [ ]:
# df = pd.read_csv("train.csv")
df = pd.read_csv("train.csv", index_col=0)
df

In [ ]:
#Filter feature data
# X_unproc = df[["LotArea","YrSold"]]
# ohe_list = ["Neighborhood", "ExterQual"]
# ohe_list = []
X_final = pd.DataFrame()

df = df.drop('Fence', axis=1)  
df = df.drop('Alley', axis=1)  
df = df.drop('PoolQC', axis=1)
df = df.drop('MiscFeature', axis=1)
df = df.drop('FireplaceQu', axis=1)

df = df.drop('LotFrontage', axis=1)  
df = df.drop('MasVnrArea', axis=1)  
df = df.drop('GarageYrBlt', axis=1)
df = df.drop('BsmtExposure', axis=1)
df = df.drop('BsmtQual', axis=1)

df = df.drop('BsmtCond', axis=1)  
df = df.drop('GarageCond', axis=1)  
df = df.drop('GarageQual', axis=1)
df = df.drop('GarageType', axis=1)
df = df.drop('GarageFinish', axis=1)

df = df.drop('BsmtFinType1', axis=1)  
df = df.drop('BsmtFinType2', axis=1)  
# df = df.drop('GarageYrBlt', axis, inplac1)
df = df.drop('Electrical', axis=1)
df = df.drop('MasVnrType', axis=1)
df = df.drop('MSSubClass', axis=1)
# Drop everything


# df["LotFrontage"] = df["LotFrontage"].fillna(df["LotFrontage"].mean())
# df["MasVnrArea"] = df["MasVnrArea"].fillna(df["MasVnrArea"].mean())
# df["GarageYrBlt"] = df["GarageYrBlt"].fillna(2001)

# df['BsmtExposure'] = df['BsmtExposure'].fillna(df['BsmtExposure'].mode()[0])
# df['BsmtQual'] = df['BsmtQual'].fillna(df['BsmtQual'].mode()[0])
# df['BsmtCond'] = df['BsmtCond'].fillna(df['BsmtCond'].mode()[0])

# df['GarageCond'] = df['GarageCond'].fillna(df['GarageCond'].mode()[0])
# df['GarageQual'] = df['GarageQual'].fillna(df['GarageQual'].mode()[0])
# df['GarageType'] = df['GarageType'].fillna(df['GarageType'].mode()[0])
# df['GarageFinish'] = df['GarageFinish'].fillna(df['GarageFinish'].mode()[0])

# df['BsmtFinType1'] = df['BsmtFinType1'].fillna(df['BsmtFinType1'].mode()[0])
# df['BsmtFinType2'] = df['BsmtFinType2'].fillna(df['BsmtFinType2'].mode()[0])

# df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])
# df['MasVnrType'] = df['MasVnrType'].fillna(df['MasVnrType'].mode()[0])

# df[(df["SalePrice"] < 200000) & (df["OverallQual"] > 8)]
# df[(df["SalePrice"] < 200000) & (df["OverallQual"] > 8) & (df["GrLivArea"] > 4000)]
# drop_index = df[(df["SalePrice"] < 200000) & (df["OverallQual"] > 8) & (df["GrLivArea"] > 4000)].index
# df = df.drop(drop_index, axis = 1)
y = df["SalePrice"].values
# df = df[["Neighborhood"]]
# df = df.drop('SalePrice', axis=1)
# df = df.drop('Id', axis=1)
print(df.columns)

df = df[["MSZoning", "LotArea", "Street", 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', "RoofStyle", 'RoofMatl','Exterior1st', 'Exterior2nd', "ExterQual"]]

print(df.values.shape)
# print(df.columns)
# df = df[['RoofMatl']]
df["RoofMatl"] = list(map(lambda roofmat: roofmat.replace("&", ""), df["RoofMatl"])) #example of remapping column values 
# df["ExterQual"] = list(map(lambda roofmat: str(roofmat.lower()) , df["ExterQual"])) #example of remapping column values 
df = df[['ExterQual', 'Exterior2nd']]

print(df[["ExterQual"]].describe())
# print(df[["ExterQual"]].isnull().sum())


In [ ]:

#Do one hot encoding for every column specified in ohe_list
for column in df.columns:
    selected_column = df[[column]]
    # print("Detected value column: " + str(selected_column.iloc[0, 0]))
    # print("Detected type column: " + str(type(selected_column.iloc[0, 0])))
    c_type = type(selected_column.iloc[0, 0])
    print(str(column) + " of type " + str(c_type))
    if c_type == str:
        # print("Detected string collumn: " + column)
        ohe_df = pd.get_dummies(selected_column, prefix=column)
        X_final = ohe_df.join(X_final)
    elif c_type == np.int16 or c_type == np.int32 or c_type == np.int64 or c_type == np.float16 or c_type == np.float32 or c_type == np.float64:
        X_final = selected_column.join(X_final)
        # print("number")
    else:
        print("Error")


        
    print(X_final)
    # print(ohe_df)

# #Do one hot encoding for every collumn specified in ohe_list
# for collumn in ohe_list:
#     pre_ohe_df = df[[collumn]]
#     ohe_df = pd.get_dummies(pre_ohe_df, prefix=collumn)
#     X_final = ohe_df.join(X_final)
#     # print(X_final)
#     # print(ohe_df)

# print(X)
# X["YrSold"] = list(map(lambda year: year - 2006, df["YrSold"])) #example of remapping column values 
# print(list(X["YrSold"]))
# X_final = X_unproc.join(X_final)
# X_final.to_csv("temp.csv")
X = X_final.values
# print(X_final.head())
X_final.to_csv("temp.csv")
print(X.shape)
# print(y.shape)
# # X = X.reshape(1460, 54)
# print(X.shape)




In [ ]:
#Split train and test data
print("X: " + str(X.shape) + " y: " + str(y.shape))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)

# X_train = X_train.reshape(-1, 2)
# print(X_train.shape)

# X_test = X_test.reshape(-1, 1)

# "x_train size: " + str(X_train.size) + " y_train size: " + str(y_train.size) 



In [ ]:
#Train LinearRegression model
reg = LinearRegression()
reg.fit(X_train, y_train)


In [ ]:
#Evaluate results
# X_test = X_test.reshape(-1, 2)
print(X_test.shape)
y_pred = reg.predict(X_test)
r_squared = reg.score(X_test, y_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)

kf = KFold(n_splits=6, shuffle=True)
cv_scores = cross_val_score(reg, X, y, cv=kf)
cv_mean = np.mean(cv_scores)

print("Mean squared error: " + str(rmse))
print("Root squared error: " + str(r_squared))
print("Mean 6-fold CV: " + str(cv_mean))




In [ ]:
#Before evaluating on history, retrieve the history

history = pd.DataFrame(columns=["Attempt", "mean_cv", "rmse"] );
try:
    history = pd.read_csv("history.csv")
    print("Found history.csv with rows amount: " + str(history.shape[0]))
except FileNotFoundError:
    print("history.csv not found, creating new one")
    history.to_csv('history.csv', index = False)


In [ ]:
#Evaluate based on history and visualize progress


history.loc[len(history.index)] = [int(len(history.index)), cv_mean, int(rmse)]
print(history[-5:-1])
print("Current attempt: ", history[-1:])
history.to_csv('history.csv', index = False)

plt.plot(history["Attempt"], history["mean_cv"])
min_cv_mean = np.min(history["mean_cv"].values)
max_cv_mean = np.max(history["mean_cv"].values)

visual_margin = (max_cv_mean - min_cv_mean) * 0.5 + 0.001 #Makes the graph more eye friendly

plt.xlabel("Attempts")
plt.xlim(0, np.size(history["Attempt"].values))
# plt.ylim(min_r_squared - visual_margin, max_r_squared + visual_margin)
plt.ylabel("Mean cross_val_score")
plt.ylim(0, 1)
plt.title("Cross-Validation score progress")
plt.show()
